In [15]:
import paramiko
import pandas as pd

In [16]:
ssh = paramiko.SSHClient()  # 创建SSH对象
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())  # 允许连接不在know_hosts文件中的主机
# ssh.connect(hostname='192.168.50.188', port=22, username='root', password='Dacheng@123')  # 连接服务器
ssh.connect(hostname='192.168.50.145', port=22, username='root', password='el{jk6ui6Q$5hThv')  # 连接服务器

### 机箱信息

In [3]:
stdin, stdout, stderr = ssh.exec_command('dmidecode -t chassis')
result_out,result_err = stdout.read(),stderr.read()  # 获取命令结果
result_out = result_out.decode('utf-8') # 解码
result_out = result_out.split('\t')
result_out

['Usage: dmidecode [OPTIONS]\nOptions are:\n -d, --dev-mem FILE     Read memory from device FILE (default: /dev/mem)\n -h, --help             Display this help text and exit\n -q, --quiet            Less verbose output\n -s, --string KEYWORD   Only display the value of the given DMI string\n -t, --type TYPE        Only display the entries of given type\n -H, --handle HANDLE    Only display the entry of given handle\n -u, --dump             Do not decode the entries\n     --dump-bin FILE    Dump the DMI data to a binary file\n     --from-dump FILE   Read the DMI data from a binary file\n     --no-sysfs         Do not attempt to read DMI data from sysfs files\n     --oem-string N     Only display the value of the given OEM string\n -V, --version          Display the version and exit\n']

In [4]:
result = []
for i in result_out:
    for j in i.split('\n'):
        if j != '':
            result.append(j)
result

['Usage: dmidecode [OPTIONS]',
 'Options are:',
 ' -d, --dev-mem FILE     Read memory from device FILE (default: /dev/mem)',
 ' -h, --help             Display this help text and exit',
 ' -q, --quiet            Less verbose output',
 ' -s, --string KEYWORD   Only display the value of the given DMI string',
 ' -t, --type TYPE        Only display the entries of given type',
 ' -H, --handle HANDLE    Only display the entry of given handle',
 ' -u, --dump             Do not decode the entries',
 '     --dump-bin FILE    Dump the DMI data to a binary file',
 '     --from-dump FILE   Read the DMI data from a binary file',
 '     --no-sysfs         Do not attempt to read DMI data from sysfs files',
 '     --oem-string N     Only display the value of the given OEM string',
 ' -V, --version          Display the version and exit']

In [25]:
information_chassis = {}
for i in result:
    if i.split(': ')[0] == 'Type':
        information_chassis['Type'] = i.split(': ')[1]
    elif i.split(': ')[0] == 'Serial Number':
        information_chassis['Serial Number'] = i.split(': ')[1]
    elif i.split(': ')[0] == 'Version':
        information_chassis['Version'] = i.split(': ')[1]
    elif i.split(': ')[0] == 'Manufacturer':
        information_chassis['Manufacturer'] = i.split(': ')[1]
information_chassis

{'Manufacturer': 'Huawei',
 'Type': 'Main Server Chassis',
 'Version': 'Huawei',
 'Serial Number': 'Huawei'}

### 系统信息

In [4]:
stdin, stdout, stderr = ssh.exec_command('dmidecode -t system')
result_out,result_err = stdout.read(),stderr.read()  # 获取命令结果
result_out = result_out.decode('utf-8') # 解码
result_out = result_out.split('\t')
result_out

['# dmidecode 3.2\nGetting SMBIOS data from sysfs.\nSMBIOS 2.8 present.\n\nHandle 0x0100, DMI type 1, 27 bytes\nSystem Information\n',
 'Manufacturer: Dell Inc.\n',
 'Product Name: PowerEdge R730\n',
 'Version: Not Specified\n',
 'Serial Number: BZLCBT2\n',
 'UUID: 4c4c4544-005a-4c10-8043-c2c04f425432\n',
 'Wake-up Type: Power Switch\n',
 'SKU Number: SKU=NotProvided;ModelName=PowerEdge R730\n',
 'Family: Not Specified\n\nHandle 0x0C00, DMI type 12, 5 bytes\nSystem Configuration Options\n',
 'Option 1: NVRAM_CLR: Clear user settable NVRAM areas and set defaults\n',
 'Option 2: PWRD_EN: Close to enable password\n\nHandle 0x2000, DMI type 32, 11 bytes\nSystem Boot Information\n',
 'Status: No errors detected\n\n']

In [5]:
result = []
for i in result_out:
    for j in i.split('\n'):
        if j != '':
            result.append(j)

In [6]:
result

['# dmidecode 3.2',
 'Getting SMBIOS data from sysfs.',
 'SMBIOS 2.8 present.',
 'Handle 0x0100, DMI type 1, 27 bytes',
 'System Information',
 'Manufacturer: Dell Inc.',
 'Product Name: PowerEdge R730',
 'Version: Not Specified',
 'Serial Number: BZLCBT2',
 'UUID: 4c4c4544-005a-4c10-8043-c2c04f425432',
 'Wake-up Type: Power Switch',
 'SKU Number: SKU=NotProvided;ModelName=PowerEdge R730',
 'Family: Not Specified',
 'Handle 0x0C00, DMI type 12, 5 bytes',
 'System Configuration Options',
 'Option 1: NVRAM_CLR: Clear user settable NVRAM areas and set defaults',
 'Option 2: PWRD_EN: Close to enable password',
 'Handle 0x2000, DMI type 32, 11 bytes',
 'System Boot Information',
 'Status: No errors detected']

In [7]:
# 输出机箱类型
for result in result_out:
    if (':'in result) and (result.split(': ')[0] == 'Manufacturer'):
        print('生产商: {}'.format(result.split(': ')[1]))
    elif (':'in result) and (result.split(': ')[0] == 'Product Name'):
        print('系统型号: {}'.format(result.split(': ')[1]))
    elif (':'in result) and (result.split(': ')[0] == 'Serial Number'):
        print('服务标签: {}'.format(result.split(': ')[1]))
    elif result.split(': ')[0] == 'Wake-up Type':
        print('网络唤醒模式: {}'.format(result.split(': ')[1]))

生产商: Dell Inc.

系统型号: PowerEdge R730

服务标签: BZLCBT2

网络唤醒模式: Power Switch



### 主板相关的硬件信息

In [11]:
stdin, stdout, stderr = ssh.exec_command('dmidecode |grep -A16 "System Information$"')
result_out,result_err = stdout.read(),stderr.read()  # 获取命令结果
result_out = result_out.decode('utf-8') # 解码
result_out = result_out.split('\t')
result_out

['System Information\n',
 'Manufacturer: Dell Inc.\n',
 'Product Name: PowerEdge R730\n',
 'Version: Not Specified\n',
 'Serial Number: BZLCBT2\n',
 'UUID: 4c4c4544-005a-4c10-8043-c2c04f425432\n',
 'Wake-up Type: Power Switch\n',
 'SKU Number: SKU=NotProvided;ModelName=PowerEdge R730\n',
 'Family: Not Specified\n\nHandle 0x0200, DMI type 2, 8 bytes\nBase Board Information\n',
 'Manufacturer: Dell Inc.\n',
 'Product Name: 072T6D\n',
 'Version: A09\n',
 'Serial Number: .BZLCBT2.CNFCP0089B0050.\n\n']

In [23]:
stdin, stdout, stderr = ssh.exec_command('dmidecode -t baseboard')
result_out,result_err = stdout.read(),stderr.read()  # 获取命令结果
result_out = result_out.decode('utf-8') # 解码
result_out = result_out.split('\t')

In [24]:
result_out

['# dmidecode 3.2\nGetting SMBIOS data from sysfs.\nSMBIOS 2.8 present.\n\nHandle 0x0200, DMI type 2, 8 bytes\nBase Board Information\n',
 'Manufacturer: Dell Inc.\n',
 'Product Name: 072T6D\n',
 'Version: A09\n',
 'Serial Number: .BZLCBT2.CNFCP0089B0050.\n\nHandle 0x2900, DMI type 41, 11 bytes\nOnboard Device\n',
 'Reference Designation: Integrated NIC 1\n',
 'Type: Ethernet\n',
 'Status: Enabled\n',
 'Type Instance: 1\n',
 'Bus Address: 0000:01:00.0\n\nHandle 0x2901, DMI type 41, 11 bytes\nOnboard Device\n',
 'Reference Designation: Integrated NIC 2\n',
 'Type: Ethernet\n',
 'Status: Enabled\n',
 'Type Instance: 2\n',
 'Bus Address: 0000:01:00.1\n\nHandle 0x2902, DMI type 41, 11 bytes\nOnboard Device\n',
 'Reference Designation: Integrated NIC 3\n',
 'Type: Ethernet\n',
 'Status: Enabled\n',
 'Type Instance: 3\n',
 'Bus Address: 0000:02:00.0\n\nHandle 0x2903, DMI type 41, 11 bytes\nOnboard Device\n',
 'Reference Designation: Integrated NIC 4\n',
 'Type: Ethernet\n',
 'Status: Enabled

### BIOS相关的硬件信息

In [25]:
stdin, stdout, stderr = ssh.exec_command('dmidecode -t bios')
result_out,result_err = stdout.read(),stderr.read()  # 获取命令结果
result_out = result_out.decode('utf-8') # 解码
result_out = result_out.split('\t')
result_out

['# dmidecode 3.2\nGetting SMBIOS data from sysfs.\nSMBIOS 2.8 present.\n\nHandle 0x0000, DMI type 0, 24 bytes\nBIOS Information\n',
 'Vendor: Dell Inc.\n',
 'Version: 2.8.0\n',
 'Release Date: 005/17/2018\n',
 'Address: 0xF0000\n',
 'Runtime Size: 64 kB\n',
 'ROM Size: 16 MB\n',
 'Characteristics:\n',
 '',
 'ISA is supported\n',
 '',
 'PCI is supported\n',
 '',
 'PNP is supported\n',
 '',
 'BIOS is upgradeable\n',
 '',
 'BIOS shadowing is allowed\n',
 '',
 'Boot from CD is supported\n',
 '',
 'Selectable boot is supported\n',
 '',
 'EDD is supported\n',
 '',
 'Japanese floppy for Toshiba 1.2 MB is supported (int 13h)\n',
 '',
 '5.25"/360 kB floppy services are supported (int 13h)\n',
 '',
 '5.25"/1.2 MB floppy services are supported (int 13h)\n',
 '',
 '3.5"/720 kB floppy services are supported (int 13h)\n',
 '',
 '8042 keyboard services are supported (int 9h)\n',
 '',
 'Serial services are supported (int 14h)\n',
 '',
 'CGA/mono video services are supported (int 10h)\n',
 '',
 'ACPI 

### 处理器相关的信息

In [115]:
stdin, stdout, stderr = ssh.exec_command('dmidecode -t processor')
result_out,result_err = stdout.read(),stderr.read()  # 获取命令结果
result_out = result_out.decode('utf-8') # 解码
result_out = result_out.split('\t')

In [116]:
result = []
for i in result_out:
    for j in i.split('\n'):
        if j != '':
            result.append(j)
result

['# dmidecode 3.2',
 'Getting SMBIOS data from sysfs.',
 'SMBIOS 2.8 present.',
 'Handle 0x0400, DMI type 4, 42 bytes',
 'Processor Information',
 'Socket Designation: CPU1',
 'Type: Central Processor',
 'Family: Xeon',
 'Manufacturer: Intel',
 'ID: F1 06 04 00 FF FB EB BF',
 'Signature: Type 0, Family 6, Model 79, Stepping 1',
 'Flags:',
 'FPU (Floating-point unit on-chip)',
 'VME (Virtual mode extension)',
 'DE (Debugging extension)',
 'PSE (Page size extension)',
 'TSC (Time stamp counter)',
 'MSR (Model specific registers)',
 'PAE (Physical address extension)',
 'MCE (Machine check exception)',
 'CX8 (CMPXCHG8 instruction supported)',
 'APIC (On-chip APIC hardware supported)',
 'SEP (Fast system call)',
 'MTRR (Memory type range registers)',
 'PGE (Page global enable)',
 'MCA (Machine check architecture)',
 'CMOV (Conditional move instruction supported)',
 'PAT (Page attribute table)',
 'PSE-36 (36-bit page size extension)',
 'CLFSH (CLFLUSH instruction supported)',
 'DS (Debug sto

In [117]:
handle_index = []
for i in result:
    if i.startswith('Handle'):
        handle_index.append(result.index(i))
handle_index

[3, 62]

In [120]:
# 分阶段处理，处理除最后一个index之外的信息
results_memory = []
for i in handle_index[:-1]:
    result_memory = []
    for j in result[i:handle_index[handle_index.index(i)+1]]:
        result_memory.append(j)
    results_memory.append(result_memory)    
result_memory = []
for j in result[handle_index[-1]:]:
    result_memory.append(j)
results_memory.append(result_memory)
results_memory

[['Handle 0x0400, DMI type 4, 42 bytes',
  'Processor Information',
  'Socket Designation: CPU1',
  'Type: Central Processor',
  'Family: Xeon',
  'Manufacturer: Intel',
  'ID: F1 06 04 00 FF FB EB BF',
  'Signature: Type 0, Family 6, Model 79, Stepping 1',
  'Flags:',
  'FPU (Floating-point unit on-chip)',
  'VME (Virtual mode extension)',
  'DE (Debugging extension)',
  'PSE (Page size extension)',
  'TSC (Time stamp counter)',
  'MSR (Model specific registers)',
  'PAE (Physical address extension)',
  'MCE (Machine check exception)',
  'CX8 (CMPXCHG8 instruction supported)',
  'APIC (On-chip APIC hardware supported)',
  'SEP (Fast system call)',
  'MTRR (Memory type range registers)',
  'PGE (Page global enable)',
  'MCA (Machine check architecture)',
  'CMOV (Conditional move instruction supported)',
  'PAT (Page attribute table)',
  'PSE-36 (36-bit page size extension)',
  'CLFSH (CLFLUSH instruction supported)',
  'DS (Debug store)',
  'ACPI (ACPI supported)',
  'MMX (MMX technol

### 内存相关的信息

In [30]:
stdin, stdout, stderr = ssh.exec_command('dmidecode -t memory')
result_out,result_err = stdout.read(),stderr.read()  # 获取命令结果
result_out = result_out.decode('utf-8') # 解码
result_out = result_out.split('\t')

### 查看网卡信息

In [5]:
stdin, stdout, stderr = ssh.exec_command("for i in seq 0 9;do ethtool eth${i} | egrep 'eth|Link';done")
result_out,result_err = stdout.read(),stderr.read()  # 获取命令结果
result_out = result_out.decode('utf-8') # 解码
result_out.split('\n')

['Settings for ethseq:', 'Settings for eth0:', 'Settings for eth9:', '']